In [ ]:
# !pip install tflearn

In [ ]:
import json

import numpy as np
import tensorflow as tf

import tflearn
import random
import pickle

import nltk
# nltk.download('all')

In [ ]:
with open('intents.json') as file:
    intents = json.load(file, strict = False) 
intents = intents['intents'] 

In [ ]:
print("[", end = "")
for intent in intents:
  print("{", end = "")
  for key, value in intent.items():
    print("{}: {},".format(key, value))
  print("\b\b\n},")
print("\b\b]")

In [ ]:
from nltk.stem.snowball import SnowballStemmer
stemmer = SnowballStemmer('english')

In [ ]:
retrain_model = True

if retrain_model:
    P_all = [] #all patterns
    T_all = [] #all tags
    patterns = [] #patterns
    tags = [] #tags
    
    for intent in intents:
        for p in intent['patterns']:
            words = nltk.word_tokenize(p)

            P_all.extend(words)
            patterns.append(words)
            tags.append(intent['tag'])
            
        T_all.append(intent['tag'])
      
 
    P_all = [stemmer.stem(word.lower()) for word in P_all]
    P_all = sorted(list(set(P_all)))
    
    T_all = sorted(T_all)
    
    x_train = []
    y_train = []
    
    y_empty = [0 for i in range(len(T_all))]
    

    for index, intent in enumerate(patterns):
        words_set = []
        
        all_words = [stemmer.stem(word.lower()) for word in intent]
        
        for word in P_all:
            if word in all_words:
                words_set.append(1)
            else:
                words_set.append(0)
                
        out = y_empty[:]
        out[T_all.index(tags[index])] = 1
        
        x_train.append(words_set)
        y_train.append(out)
    
    x_train = np.array(x_train)
    y_train = np.array(y_train)
    
    with open('training_data.pickle', 'wb') as f:
        pickle.dump((P_all, T_all, x_train, y_train), f)
else:
    with open('training_data.pickle', 'rb') as f:
        P_all, T_all, x_train, y_train = pickle.load(f)

In [ ]:
# !pip install tensorflow==1.15

In [ ]:
# tf.reset_default_graph()
from tensorflow.python.framework import ops
ops.reset_default_graph()

nn = tflearn.input_data(shape = [None, len(x_train[0])])
nn = tflearn.fully_connected(nn, 8)
nn = tflearn.fully_connected(nn, 8)

nn = tflearn.fully_connected(nn, len(y_train[0]), activation = 'softmax')
nn = tflearn.regression(nn)

model = tflearn.DNN(nn)

In [ ]:
if retrain_model:
    model.fit(x_train, y_train, n_epoch = 1000, batch_size = 8, show_metric = True)
    model.save('model.tfl')
else:
    model.load('./model.tfl')

In [ ]:
def text_set(t, P_all):
    words_set = [0 for i in range(len(P_all))]
    
    text = nltk.word_tokenize(t)
    text = [stemmer.stem(word.lower()) for word in text]
    
    for word in text:
        if word in P_all:
            words_set[P_all.index(word)] = 1
    
    return np.array(words_set)

In [ ]:
def chat():
    print("Enter a message to talk to the bot [type stop to exit].")
    
    context = None
    
    default = ['Sorry, Im not sure I know what you mean! You could try rephrasing that!',
                         'I dont understand that! Try rephrasing it.']

    while True:
        user = str(input('You: '))
        if user.lower() == 'stop':
            break
        
        user_set = text_set(user, P_all)

        res = model.predict([user_set])[0]

        res_index = np.argmax(res)
        res_tag = T_all[res_index]
        

        if res[res_index] > 0.8:
            for intent in intents:
                if intent['tag'] == res_tag:
                    if 'context_filter' not in intent or 'context_filter' in intent and intent['context_filter'] == context:
                        outcomes = intent['responses']
                        if 'context_set' in intent:
                            context = intent['context_set']
                        else:
                            context = None
                        print(random.choice(outcomes))
                    else:
                        print(random.choice(default))
        else:
            print(random.choice(default))

In [ ]:
chat()